In [7]:
%cd /kaggle/working/
%rm -rf mlproduction
!git clone https://github.com/daoanhkhoa123/mlproduction.git
%cd /kaggle/working/mlproduction
!ls

/kaggle/working
Cloning into 'mlproduction'...
remote: Enumerating objects: 111, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 111 (delta 55), reused 92 (delta 36), pack-reused 0 (from 0)
Receiving objects: 100% (111/111), 3.33 MiB | 11.47 MiB/s, done.
Resolving deltas: 100% (55/55), done.
/kaggle/working/mlproduction
main.py  pyproject.toml  README.md  runtime  scripts  uv.lock


# Setup

In [8]:
!chmod +x scripts/cloud_setup.sh
!scripts/cloud_setup.sh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 75.4 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: uv
    Found existing installation: uv 0.9.17
    Uninstalling uv-0.9.17:
      Successfully uninstalled uv-0.9.17
Using CPython 3.12.12
Creating virtual environment at: .venv
Resolved 36 packages in 1ms
░░░░░░░░░░░░░░░░░░░░ [0/30] Installing wheels...                                warning: Failed to hardlink files; falling back to full copy. This may lead to degraded performance.
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 30 packages in 3.36s                              
 + annotated-types==0.7.0
 + filelock==3.20.0
 + fsspec==2025.12.0
 + jinja2==3.1.6
 + markupsafe==3.0.2
 + mpmath==1.3.0
 + networkx==3.6.1
 + nvidia-cublas-cu12==12.4.5.8
 + nvidia-cuda-cupti-cu12==12.4.127
 + nvi